# Ensemble model calibration using a spatiotemporal proxy for CO2 Monitoring
### Misael M. Morales, The University of Texas at Austin
Morales, M.M., Torres-Verdin, C., and Pyrcz, M. J. (2024). Ensemble model calibration and uncertainty quantification in geologic CO2 storage using a spatiotemporal deep learning proxy

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from time import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import scipy.io as sio
import scipy.linalg as sla
import scipy.optimize as sopt
import scipy.spatial as sptl
from pyesmda import ESMDA, ESMDA_RS, approximate_cov_mm

import tensorflow as tf
from utils import check_tf_gpu, describe_data, load_data
from pyesmda import ESMDA, ESMDA_RS, approximate_cov_mm

NREALIZATIONS = 1000
NX, NY = 64, 64
NTT, NT = 28, 20
NTRAIN = 700
NVALID = 150
BATCHSIZE = 16
HIDDEN = [16, 64, 256]

sec2year   = 365.25 * 24 * 60 * 60
Darcy      = 9.869233e-13
psi2pascal = 6894.76
co2_rho    = 686.5266
milli      = 1e-3
mega       = 1e6
check_tf_gpu()

--------------------------------------------------------------
------------------------ VERSION INFO ------------------------
TF version: 2.15.0 | Keras: 3.5.0 | # Device(s) available: 2
TF Built with CUDA? True | CUDA: 12.2 | cuDNN: 8
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
--------------------------------------------------------------



In [2]:
all_data = np.load('data/simulations_64x64x28.npz')
poro = all_data['poro']
perm = all_data['perm']
pressure = all_data['pressure']
saturation = all_data['saturation']
print('poro: {} | perm: {}'.format(poro.shape, perm.shape))
print('pressure: {} | saturation: {}'.format(pressure.shape, saturation.shape))

pmin, pmax = poro.min(), poro.max()
kmin, kmax = perm.min(), perm.max()
rmin, rmax = pressure.min(), pressure.max()
smin, smax = saturation.min(), saturation.max()

poro_norm = (poro - pmin) / (pmax - pmin)
perm_norm = (perm - kmin) / (kmax - kmin)
pressure_norm = (pressure - rmin) / (rmax - rmin)
saturation_norm = (saturation - smin) / (smax - smin)

features = np.stack([poro_norm, perm_norm], -1)
targets  = np.stack([pressure_norm, saturation_norm], -1)
print('features: {} | targets: {}'.format(features.shape, targets.shape))

poro: (1000, 64, 64) | perm: (1000, 64, 64)
pressure: (1000, 28, 64, 64) | saturation: (1000, 28, 64, 64)
features: (1000, 64, 64, 2) | targets: (1000, 28, 64, 64, 2)


In [3]:
idx = np.random.choice(range(NREALIZATIONS), NREALIZATIONS, replace=False)
train_idx, test_idx = idx[:NTRAIN], idx[NTRAIN:]

X_train, X_test = features[train_idx], features[test_idx]
y_train, y_test = targets[train_idx], targets[test_idx]

y1_train, y2_train = y_train[:,:NTT-NT], y_train[:,NTT-NT:]
y1_test, y2_test = y_test[:,:NTT-NT], y_test[:,NTT-NT:]

print('Train - X: {} | y1: {} | y2: {}'.format(X_train.shape, y1_train.shape, y2_train.shape))
print('Test  - X: {} | y1: {} | y2: {}'.format(X_test.shape, y1_test.shape, y2_test.shape))

Train - X: (700, 64, 64, 2) | y1: (700, 8, 64, 64, 2) | y2: (700, 20, 64, 64, 2)
Test  - X: (300, 64, 64, 2) | y1: (300, 8, 64, 64, 2) | y2: (300, 20, 64, 64, 2)


ValueError: Array has shape (None, 8, 256) but out-of-bounds index (Ellipsis, 8, slice(None, None, None)) was requested.

***
# END